# Objetivo

Este simples pipeline foi desenvolvido para monitoramento dos casos de COVID atráves de uma API pública.
Os dados são coletados e armazenados em um database para que posteriormente os dados sejam cosumidos em uma ferramenta de BI.

# Fonte de dados
Os dados sobre o COVID são totalmente tiradas da API publica > https://covid19-brazil-api.now.sh/api/report/v1

# Importando as bibliotecas

In [73]:
import requests as r
import pandas as pd
from datetime import datetime
from pandasql import sqldf
import pyodbc as odc

# Variaveis Globais

In [74]:
STRING_CONEX_BANCO = 'DRIVER={SQL Server};''SERVER=DESKTOP-AF7BA3M;''DATABASE=CovidDB;'

# Requisitando os dados na API

In [75]:
url = 'https://covid19-brazil-api.now.sh/api/report/v1'
df = r.request('GET',url)

# Convertendo o Json em um dataframe

In [76]:
#Obtendo a data atual do sistema.
data_atual = datetime.today().strftime('%Y-%m-%d')

df1 = pd.DataFrame(df.json()['data'])

#Transformando o campo datetime
df1['datetime'] = pd.to_datetime(df1['datetime']).dt.strftime("%Y-%m-%d")

In [77]:
#Visualizando as 5 primeiras linhas do dataframe
df1.head()

,uid,uf,state,cases,deaths,suspects,refuses,datetime
0,35,SP,São Paulo,5434438,168466,5334,596,2022-05-11
1,31,MG,Minas Gerais,3366689,61402,925,104,2022-05-11
2,41,PR,Paraná,2478462,43158,400,119,2022-05-11
3,43,RS,Rio Grande do Sul,2363227,39361,416,330,2022-05-11
4,33,RJ,Rio de Janeiro,2158019,73616,1254,148,2022-05-11


In [78]:
#Visualizando as 5 ultimas linhas do DataFrame
df1.tail()

,uid,uf,state,cases,deaths,suspects,refuses,datetime
22,17,TO,Tocantins,304931,4156,13,0,2022-05-11
23,27,AL,Alagoas,298496,6935,43,22,2022-05-11
24,16,AP,Amapá,160401,2132,15,0,2022-05-11
25,14,RR,Roraima,155582,2151,9,0,2022-05-11
26,12,AC,Acre,124969,2002,12,0,2022-05-11


In [79]:
# Verificando se temos dados ausentes no dataframe.
print(" --> Dados API")
display(df1.isnull().sum())

 --> Dados API


uid         0
uf          0
state       0
cases       0
deaths      0
suspects    0
refuses     0
datetime    0
dtype: int64

# Select no bando de dados SqlServer

O retorno do select será usado em uma função de validação.

In [80]:
def SelectBanco (STRING_CONEX_BANCO):
    
    cnxn = odc.connect(STRING_CONEX_BANCO)
    query = """SELECT * FROM Tb_Covid1"""
    
    #Passando o retorno da consulta para um dataframe = 'resultado'
    resultado = pd.read_sql(query, cnxn)
    
    return resultado

# Função de validação e chamada da função  GravarNoBanco()

In [95]:
def IncrementoBanco (query,data_atual,df1,STRING_CONEX_BANCO):
    df0 = pd.DataFrame(query)
    
    try:
        if (df1['datetime'].max() != df0['datetime'].max()) :
            GravarNoBanco(df1,STRING_CONEX_BANCO) 
            print("Os dados foram inseridos no banco. Qty de rows", df1.shape)
        else: 
            print("Já existem dados no banco com a data -->", df1['datetime'][0])
    except:
        print("ALERTA --> Não foi possivel inserir os dados no banco <-- ")
        

# Função de incremento no Banco de Dados

In [93]:
def GravarNoBanco(df,STRING_CONEX_BANCO):

        cnxn = odc.connect(STRING_CONEX_BANCO)
        cursor = cnxn.cursor()

        for index , linha in df.iterrows():
            cursor.execute("insert into Tb_Covid1(uid, uf, state,cases,deaths,suspects,refuses,datetime) values (?,?,?,?,?,?,?,?)",linha.uid,linha.uf,linha.state,linha.cases,linha.deaths,linha.suspects,linha.refuses,linha.datetime)
            cursor.commit()

# Instancioando as funçoes

In [94]:
# Query no banco de dados com o objetivo de verificar o registro com a maior data.
query = SelectBanco(STRING_CONEX_BANCO)

#Incrementando os dados no banco
IncrementoBanco(query,data_atual,df1,STRING_CONEX_BANCO)

Já existem dados no banco com a data --> 2022-05-11 (27, 8)


# Consultando o banco para verificar os valores inseridos.

In [97]:
query = SelectBanco(STRING_CONEX_BANCO)
query.head(50)
query['datetime'].max()

'2022-05-11'